## Pyvis, Gephi

In [3]:
import sys
sys.path.append("..")
from rcn_py import visual

In [26]:
visual.pyvis_network_by_datafile('Stefan Verhoeven', "/Users/jennifer/RCSNA/data", "/Users/jennifer/RCSNA/visualization")

/Users/jennifer/RCSNA/rcn-py/notebook/../rcn_py/data_storage.py:206: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  authors_orcid[i] = authors_orcid[i].strip('[')
/Users/jennifer/RCSNA/rcn-py/notebook/../rcn_py/data_storage.py:207: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  authors_orcid[i] = authors_orcid[i].strip(']')
/Users/jennifer/RCSNA/rcn-py/notebook/../rcn_py/data_storage.py:208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  

In [4]:
visual.gephi_network_by_datafile('Stefan Verhoeven', "/Users/jennifer/RCSNA/data", "/Users/jennifer/RCSNA/visualization")

/Users/jennifer/RCSNA/rcn-py/notebook/../rcn_py/data_storage.py:206: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  authors_orcid[i] = authors_orcid[i].strip('[')
/Users/jennifer/RCSNA/rcn-py/notebook/../rcn_py/data_storage.py:207: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  authors_orcid[i] = authors_orcid[i].strip(']')
/Users/jennifer/RCSNA/rcn-py/notebook/../rcn_py/data_storage.py:208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  

## Neo4j to D3

In [2]:
from json import dumps
import logging
import os
from flask import (
    Flask,
    g,
    request,
    Response,
)
from neo4j import (
    GraphDatabase,
    basic_auth,
)

In [5]:
uri = "bolt://localhost:7687"
username = "neo4j"
password = "zhiningbai"

In [8]:
# app = Flask(__name__, static_url_path="/static/")

url = os.getenv("NEO4J_URI", uri)
# username = os.getenv("NEO4J_USER", "movies")
# password = os.getenv("NEO4J_PASSWORD", "movies")
neo4j_version = os.getenv("NEO4J_VERSION", "4")
database = os.getenv("NEO4J_DATABASE", "RCN DB")

# port = os.getenv("PORT", 8080)

driver = GraphDatabase.driver(url, auth=basic_auth(username, password))

In [ ]:
def get_db():
    if not hasattr(g, "neo4j_db"):
        if neo4j_version >= "4":
            g.neo4j_db = driver.session(database=database)
        else:
            g.neo4j_db = driver.session()
    return g.neo4j_db

def close_db(error):
    if hasattr(g, "neo4j_db"):
        g.neo4j_db.close()